Task 3: -
In this task the problem was to predict the winner of world cup 2023!! Now to address this task there were several problems listed below, and I tried to solve these problems one by one till the final problem of predicting the winner of world cup is solved.
Problem Faced: -
1)	Which dataset should be used?
2)	How to preprocesses the data such that the model learns accurately?
3)	Which model to select and which hyperparameters should be chosen in order to maximise the accuracy, precision and recall of the classification model
Solution Used: -
-	To solve the first problem, I first done the training and testing only dataset having 3 features team 1, team 2 and ground and the target column was winner. Here I trained and tested the random forest classifier on the selected dataset. Before training and testing I one hot encoded the team1,team2 and label encoded the ground column, and for the target variable I binarized it like if the first team is winning then the value will be assigned as 0 and if second teams wins 1 is assigned. The random forest classifier gave 60% testing accuracy and predicted that England will win the world cup.
-	Then I tried to add more features, namely NRR(net run rate, both the team), Points(both the team) and which team is batting first. On this dataset I trained several models
-	Preprocessing: - In preprocessing step I label encoded team 1, team 2 and ground then I encoded the winner team in binarized manner as discussed above and also label encoded the winner team and trained all the models on this 2 encodings(target column encoded as binary encoding and label encoding.)
-	Implemented models: - 1) XGboost, 2) Catboost, 3) LGBMclassifier, 4) DNN(having 4 hidden layers)
-	In all the models DNN(deep neural network) and LGBM classifiers outperforms all the model with accuracy of 72.73% testing accuracy and above 98% training accuracy. Both the model predicts INDIA is going to win the world cup.
-	In the DNN I used callbacks to save the best model parameters which gives highest validation accuracy also, used dropouts to reduce overfitting. Then in model.compile I used adam as optimizer and for binary encoded winner case loss function is binary_crossentropy and in the case where winner is encoded as label categorical_crossentropy is used.
-	The model is trained on batch_size 32 and 100 epochs and gives 72.73% accuracy on tesing dataset.

Hyperparameter tuning
-	In hyperparameter tuning used grid search to find the optimal parameter of the choosed model.The results are shown in the google file 202101102_course_project_3.

# **Imports**

In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report

# **Task 3 : - Approach 1**

In [ ]:
import pandas as pd

In [ ]:
!unzip /content/drive/Shareddrives/Repository/data_mining_project/world_cup_2023_dataset_kaggle/archive_1.zip -d /content/drive/Shareddrives/Repository/data_mining_project/world_cup_2023_dataset_kaggle

Archive:  /content/drive/Shareddrives/Repository/data_mining_project/world_cup_2023_dataset_kaggle/archive_1.zip
  inflating: /content/drive/Shareddrives/Repository/data_mining_project/world_cup_2023_dataset_kaggle/Fixtures.csv  
  inflating: /content/drive/Shareddrives/Repository/data_mining_project/world_cup_2023_dataset_kaggle/Icc_ranking.csv  
  inflating: /content/drive/Shareddrives/Repository/data_mining_project/world_cup_2023_dataset_kaggle/World_cup_2023.csv  
  inflating: /content/drive/Shareddrives/Repository/data_mining_project/world_cup_2023_dataset_kaggle/results.csv  


In [ ]:
results = pd.read_csv('/content/drive/Shareddrives/Repository/data_mining_project/world_cup_2023_dataset_kaggle/results.csv')

In [ ]:
worldcup_teams = ['England', ' South Africa', '', 'West Indies',
            'Pakistan', 'New Zealand', 'Sri Lanka', 'Afghanistan',
            'Australia', 'Bangladesh', 'India']


df_team_1 = results[results['Team_1'].isin(worldcup_teams)]

df_team_2 = results[results['Team_2'].isin(worldcup_teams)]

df_teams = pd.concat((df_team_1,df_team_2))

df_teams.drop_duplicates()

df_teams.count()

Date      801
Team_1    801
Team_2    801
Winner    801
Margin    801
Ground    801
dtype: int64

In [ ]:
df_teams_2023 = df_teams.drop(['Margin'],axis=1)

In [ ]:
df_teams_2023

,Date,Team_1,Team_2,Winner,Ground
8,12/06/2015,England,New Zealand,New Zealand,Kennington Oval
11,18/06/2015,Bangladesh,India,Bangladesh,Shere Bangla National Stadium
35,08/09/2015,England,Australia,England,Old Trafford
53,07/11/2015,Sri Lanka,West Indies,Sri Lanka,Pallekele International Cricket Stadium
56,11/11/2015,Bangladesh,Zimbabwe,Bangladesh,Shere Bangla National Stadium
...,...,...,...,...,...
716,05/07/2023,Bangladesh,Afghanistan,Afghanistan,Zahur Ahmed Chowdhury Stadium
718,07/07/2023,Sri Lanka,West Indies,Sri Lanka,Harare Sports Club
719,08/07/2023,Bangladesh,Afghanistan,Afghanistan,Zahur Ahmed Chowdhury Stadium
720,09/07/2023,Netherlands,Sri Lanka,Sri Lanka,Harare Sports Club


In [ ]:
df_teams_2023 = df_teams_2023.reset_index(drop=True)

df_teams_2023.loc[df_teams_2023.Winner == df_teams_2023.Team_1,'winning_team'] = 1

df_teams_2023.loc[df_teams_2023.Winner == df_teams_2023.Team_2, 'winning_team']=2

df_teams_2023.head()

,Date,Team_1,Team_2,Winner,Ground,winning_team
0,12/06/2015,England,New Zealand,New Zealand,Kennington Oval,NaN
1,18/06/2015,Bangladesh,India,Bangladesh,Shere Bangla National Stadium,NaN
2,08/09/2015,England,Australia,England,Old Trafford,1.0
3,07/11/2015,Sri Lanka,West Indies,Sri Lanka,Pallekele International Cricket Stadium,1.0
4,11/11/2015,Bangladesh,Zimbabwe,Bangladesh,Shere Bangla National Stadium,1.0


In [ ]:
df_teams_2023['Winning_team'] = df_teams_2023['Winner'].apply(lambda x: 1 if x == 'Team_1' else 2)

In [ ]:
df_teams_2023.drop(['winning_team'],axis=1,inplace=True)

In [ ]:
df_teams_2023.drop(['Winning_team'],axis=1,inplace=True)

In [ ]:
df_teams_2023['Date'] = pd.to_datetime(df_teams_2023['Date'])

# Extract the year component
df_teams_2023['Date'] = df_teams_2023['Date'].dt.year

df_teams_2023['Date']

<ipython-input-13-b88596fed5ab>:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_teams_2023['Date'] = pd.to_datetime(df_teams_2023['Date'])


0      2015
1      2015
2      2015
3      2015
4      2015
       ... 
796    2023
797    2023
798    2023
799    2023
800    2023
Name: Date, Length: 801, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# Fit and transform the categorical column
df_teams_2023['Ground'] = label_encoder.fit_transform(df_teams_2023['Ground'])

In [ ]:
from sklearn.preprocessing import StandardScaler

columns_to_scale = ['Date', 'Ground']

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit and transform the selected columns
df_teams_2023[columns_to_scale] = scaler.fit_transform(df_teams_2023[columns_to_scale])

In [ ]:
df_teams_2023.head()

,Date,Team_1,Team_2,Winner,Ground
0,-1.667849,England,New Zealand,New Zealand,-0.597793
1,-1.667849,Bangladesh,India,Bangladesh,0.910645
2,-1.667849,England,Australia,England,0.059731
3,-1.667849,Sri Lanka,West Indies,Sri Lanka,0.098409
4,-1.667849,Bangladesh,Zimbabwe,Bangladesh,0.910645


In [ ]:
from sklearn.model_selection import train_test_split

final = pd.get_dummies(df_teams_2023, prefix=['Team_1', 'Team_2'], columns=['Team_1', 'Team_2'])

X = final.drop(['Winner'], axis=1)  # Extracting features by dropping the 'Winner' column
y = final["Winner"]  # Assigning the 'Winner' column as the target variable

# Split the data into training and testing sets using a test size of 20% and a random state of 42.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

from sklearn.ensemble import RandomForestClassifier

# Create an Random forest classifier with a linear kernel
rf = RandomForestClassifier(n_estimators=100, max_depth=20,
                              random_state=0)

# Fit the classifier on the training data
rf.fit(X_train, y_train)

# Make predictions on the test data
pred = rf.predict(X_test)

# Calculating training set score:
score = rf.score(X_train, y_train)

# Calculating test set score:
score2 = rf.score(X_test, y_test)

# Printing the training set accuracy with 3 decimal places
print("Training set accuracy: ", '%.3f'%(score))

# Printing the test set accuracy with 3 decimal places
print("Test set accuracy: ", '%.3f'%(score2))

Training set accuracy:  0.909
Test set accuracy:  0.609


In [ ]:
df_teams_2023_new = df_teams_2023.drop(['Date','Ground'],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

final = pd.get_dummies(df_teams_2023_new, prefix=['Team_1', 'Team_2'], columns=['Team_1', 'Team_2'])

X = final.drop(['Winner'], axis=1)  # Extracting features by dropping the 'Winner' column
y = final["Winner"]  # Assigning the 'Winner' column as the target variable

# Split the data into training and testing sets using a test size of 20% and a random state of 42.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

from sklearn.ensemble import RandomForestClassifier

# Create an Random forest classifier with a linear kernel
rf = RandomForestClassifier(n_estimators=100, max_depth=20,
                              random_state=0)

# Fit the classifier on the training data
rf.fit(X_train, y_train)

# Make predictions on the test data
pred = rf.predict(X_test)

# Calculating training set score:
score = rf.score(X_train, y_train)

# Calculating test set score:
score2 = rf.score(X_test, y_test)

# Printing the training set accuracy with 3 decimal places
print("Training set accuracy: ", '%.3f'%(score))

# Printing the test set accuracy with 3 decimal places
print("Test set accuracy: ", '%.3f'%(score2))

Training set accuracy:  0.706
Test set accuracy:  0.609


# **Approach 2: - habing NRR,Pts and batting sequence**

In [ ]:
import pandas as pd

In [ ]:
df_icc = pd.read_csv('/content/drive/Shareddrives/Repository/data_mining_project/world_cup_2023_dataset_kaggle/results.csv')

In [ ]:
df_icc.head()

,Date,Team_1,Team_2,Winner,Margin,Ground
0,17/04/2015,Bangladesh,Pakistan,Bangladesh,won by 79 runs,Shere Bangla National Stadium
1,19/04/2015,Bangladesh,Pakistan,Bangladesh,won by 7 wickets,Shere Bangla National Stadium
2,22/04/2015,Bangladesh,Pakistan,Bangladesh,won by 8 wickets,Shere Bangla National Stadium
3,08/05/2015,Ireland,England,No result,No result,The Village
4,26/05/2015,Pakistan,Zimbabwe,Pakistan,won by 41 runs,Gaddafi Stadium


In [ ]:
df_icc['Date'] = pd.to_datetime(df_icc['Date'], format='%d/%m/%Y')

# Extract the year and create a new column
df_icc['Year'] = df_icc['Date'].dt.year

In [ ]:
df_icc['Year'].value_counts()

2019    127
2017    114
2018    101
2016     92
2022     89
2015     65
2023     54
2021     48
2020     32
Name: Year, dtype: int64

In [ ]:
selected_years = [2023, 2015, 2019]
filtered_df_icc = df_icc[df_icc['Year'].isin(selected_years)]
filtered_df_icc.head()

,Date,Team_1,Team_2,Winner,Margin,Ground,Year
0,2015-04-17,Bangladesh,Pakistan,Bangladesh,won by 79 runs,Shere Bangla National Stadium,2015
1,2015-04-19,Bangladesh,Pakistan,Bangladesh,won by 7 wickets,Shere Bangla National Stadium,2015
2,2015-04-22,Bangladesh,Pakistan,Bangladesh,won by 8 wickets,Shere Bangla National Stadium,2015
3,2015-05-08,Ireland,England,No result,No result,The Village,2015
4,2015-05-26,Pakistan,Zimbabwe,Pakistan,won by 41 runs,Gaddafi Stadium,2015


In [ ]:
filtered_df_icc.drop(['Date'],axis=1,inplace=True)
filtered_df_icc['Year'].value_counts()

<ipython-input-10-4acb5abc95f6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_icc.drop(['Date'],axis=1,inplace=True)


2019    127
2015     65
2023     54
Name: Year, dtype: int64

In [ ]:
selected_values = ['India','South Africa','Australia','New Zealand','Pakistan','Afghanistan','England','Bangladesh','Sri Lanka','Netherlands']

filtered_df_icc = filtered_df_icc[filtered_df_icc['Team_1'].isin(selected_values)]

filtered_df_icc = filtered_df_icc[filtered_df_icc['Team_2'].isin(selected_values)]

In [ ]:
filtered_df_icc['Team_1'].value_counts()

Pakistan        13
India            9
England          7
Bangladesh       7
South Africa     6
New Zealand      6
Sri Lanka        6
Netherlands      2
Afghanistan      1
Australia        1
Name: Team_1, dtype: int64

In [ ]:
filtered_df_icc = filtered_df_icc[~filtered_df_icc['Margin'].str.contains('Match abandoned')]

In [ ]:
filtered_df_icc['Margin'] = filtered_df_icc['Margin'].str.split().str[-1]

filtered_df_icc['Margin'].value_counts()

runs       35
wickets    19
tied        1
Name: Margin, dtype: int64

In [ ]:
filtered_df_icc = filtered_df_icc.rename(columns={'Margin': 'Batting'})

In [ ]:
mapping = {'runs': 1, 'wickets': 2, 'tied': 0}

# Use replace to map values in 'batting_column' to new values
filtered_df_icc['Batting'] = filtered_df_icc['Batting'].replace(mapping)

filtered_df_icc['Batting'].value_counts()

1    35
2    19
0     1
Name: Batting, dtype: int64

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
csv_path = "/content/drive/Shareddrives/Repository/data_mining_project/icc_worldcup_11_date.csv"

filtered_df_icc.to_csv(csv_path,index=False)

In [ ]:
filtered_df_icc = pd.read_csv('/content/drive/Shareddrives/Repository/data_mining_project/icc_worldcup_11_date.csv')

In [ ]:
filtered_df_icc.head()

,Team_1,Team_2,Winner,Batting,Ground,Year
0,England,New Zealand,New Zealand,1,Kennington Oval,2015
1,Bangladesh,India,Bangladesh,1,Shere Bangla National Stadium,2015
2,South Africa,New Zealand,South Africa,1,SuperSport Park,2015
3,England,Australia,England,1,Old Trafford,2015
4,Pakistan,England,England,1,Dubai International Cricket Stadium,2015


In [ ]:
wc_2015 = pd.read_csv('/content/drive/Shareddrives/Repository/data_mining_project/world_cup_dataset_2015.csv')
wc_2019 = pd.read_csv('/content/drive/Shareddrives/Repository/data_mining_project/world_cup_dataset _2019.csv')
wc_2023 = pd.read_csv('/content/drive/Shareddrives/Repository/data_mining_project/world_cup_dataset_2023.csv')

In [ ]:
filtered_df_icc_2015 =  filtered_df_icc[filtered_df_icc['Year']==2015]
filtered_df_icc_2019 =  filtered_df_icc[filtered_df_icc['Year']==2019]
filtered_df_icc_2023 =  filtered_df_icc[filtered_df_icc['Year']==2023]

In [ ]:
wc_2015.columns

Index(['Team', 'NRR', 'Pts'], dtype='object')

In [ ]:
wc_2023_team_2 = wc_2023.rename(columns={'Team': 'Team_2','NRR': 'Team_2_NRR', 'Pts': 'Team_2_Pts'})

merge_2023_team2 = pd.merge(filtered_df_icc_2023, wc_2023_team_2, on='Team_2', how='inner')

merge_2023_team2.head()

,Team_1,Team_2,Winner,Batting,Ground,Year,Team_2_NRR,Team_2_Pts
0,Pakistan,New Zealand,Pakistan,2,National Stadium (Karachi),2023,0.743,10
1,Pakistan,New Zealand,New Zealand,1,National Stadium (Karachi),2023,0.743,10
2,India,New Zealand,India,1,Rajiv Gandhi International Stadium,2023,0.743,10
3,India,New Zealand,India,2,Shaheed Veer Narayan Sing International Stadium,2023,0.743,10
4,India,New Zealand,India,1,Holkar Cricket Stadium,2023,0.743,10


In [ ]:
wc_2023_team_1 = wc_2023.rename(columns={'Team': 'Team_1','NRR': 'Team_1_NRR', 'Pts': 'Team_1_Pts'})

merge_2023_team1 = pd.merge(filtered_df_icc_2023, wc_2023_team_1, on='Team_1', how='inner')

merge_2023_team1.head()

,Team_1,Team_2,Winner,Batting,Ground,Year,Team_1_NRR,Team_1_Pts
0,Pakistan,New Zealand,Pakistan,2,National Stadium (Karachi),2023,0.036,8
1,Pakistan,New Zealand,New Zealand,1,National Stadium (Karachi),2023,0.036,8
2,Pakistan,New Zealand,Pakistan,2,Rawalpindi Cricket Stadium,2023,0.036,8
3,Pakistan,New Zealand,Pakistan,2,Rawalpindi Cricket Stadium,2023,0.036,8
4,Pakistan,New Zealand,Pakistan,1,National Stadium (Karachi),2023,0.036,8


In [ ]:
df = merge_2023_team2[['Team_2_NRR','Team_2_Pts']]

In [ ]:
result_df_2023 = pd.concat([merge_2023_team1, df], axis=1)

In [ ]:
result_df_2015.head()

,Team_1,Team_2,Winner,Batting,Ground,Year,Team_1_NRR,Team_1_Pts,Team_2_NRR,Team_2_Pts
0,England,New Zealand,New Zealand,1,Kennington Oval,2015,-0.753,4.0,2.564,12.0
1,England,Australia,England,1,Old Trafford,2015,-0.753,4.0,2.564,12.0
2,Bangladesh,India,Bangladesh,1,Shere Bangla National Stadium,2015,0.136,7.0,1.827,12.0
3,South Africa,New Zealand,South Africa,1,SuperSport Park,2015,1.707,8.0,2.257,9.0
4,Pakistan,England,England,1,Dubai International Cricket Stadium,2015,-0.085,8.0,-0.753,4.0


In [ ]:
result_df_2019.head()


,Team_1,Team_2,Winner,Batting,Ground,Year,Team_1_NRR,Team_1_Pts,Team_2_NRR,Team_2_Pts
0,New Zealand,Sri Lanka,New Zealand,1,Bay Oval,2019,0.175,11.0,-0.919,8.0
1,New Zealand,Bangladesh,New Zealand,2,Hagley Oval,2019,0.175,11.0,-0.919,8.0
2,New Zealand,South Africa,New Zealand,2,Edgbaston,2019,0.175,11.0,-0.919,8.0
3,South Africa,Pakistan,South Africa,1,SuperSport Park,2019,-0.030,7.0,-0.430,11.0
4,Pakistan,Australia,Australia,1,Sheikh Zayed Stadium,2019,-0.430,11.0,-0.430,11.0


In [ ]:
result_df_2023.head()

,Team_1,Team_2,Winner,Batting,Ground,Year,Team_1_NRR,Team_1_Pts,Team_2_NRR,Team_2_Pts
0,Pakistan,New Zealand,Pakistan,2,National Stadium (Karachi),2023,0.036,8,0.743,10
1,Pakistan,New Zealand,New Zealand,1,National Stadium (Karachi),2023,0.036,8,0.743,10
2,Pakistan,New Zealand,Pakistan,2,Rawalpindi Cricket Stadium,2023,0.036,8,0.743,10
3,Pakistan,New Zealand,Pakistan,2,Rawalpindi Cricket Stadium,2023,0.036,8,0.743,10
4,Pakistan,New Zealand,Pakistan,1,National Stadium (Karachi),2023,0.036,8,0.743,10


In [ ]:
final_world_cup = pd.concat([result_df_2015, result_df_2019, result_df_2023], axis=0, ignore_index=True)

# **Saving the CSV file**

In [ ]:
csv_path = "/content/drive/Shareddrives/Repository/data_mining_project/final_icc_worldcup_11_date.csv"

final_world_cup.to_csv(csv_path,index=False)

In [ ]:
final_world_cup.head()

,Team_1,Team_2,Winner,Batting,Ground,Year,Team_1_NRR,Team_1_Pts,Team_2_NRR,Team_2_Pts
0,England,New Zealand,New Zealand,1,Kennington Oval,2015,-0.753,4.0,2.564,12.0
1,England,Australia,England,1,Old Trafford,2015,-0.753,4.0,2.564,12.0
2,Bangladesh,India,Bangladesh,1,Shere Bangla National Stadium,2015,0.136,7.0,1.827,12.0
3,South Africa,New Zealand,South Africa,1,SuperSport Park,2015,1.707,8.0,2.257,9.0
4,Pakistan,England,England,1,Dubai International Cricket Stadium,2015,-0.085,8.0,-0.753,4.0


In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# Fit and transform the specified column
final_world_cup['Ground_encoded'] = label_encoder.fit_transform(final_world_cup['Ground'])

In [ ]:
final_world_cup.drop(['Ground'],axis=1,inplace=True)

In [ ]:
final_world_cup['Team_1_encoded'] = label_encoder.fit_transform(final_world_cup['Team_1'])

In [ ]:
mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Mapping from categorical values to label-encoded values:")
print(mapping)

Mapping from categorical values to label-encoded values:
{'Afghanistan': 0, 'Australia': 1, 'Bangladesh': 2, 'England': 3, 'India': 4, 'Netherlands': 5, 'New Zealand': 6, 'Pakistan': 7, 'South Africa': 8, 'Sri Lanka': 9}


In [ ]:
mapping = {'Afghanistan': 0, 'Australia': 1, 'Bangladesh': 2, 'England': 3, 'India': 4, 'Netherlands': 5, 'New Zealand': 6, 'Pakistan': 7, 'South Africa': 8, 'Sri Lanka': 9}

final_world_cup['Team_2_encoded'] = final_world_cup['Team_2'].replace(mapping)
final_world_cup['winner_encoded'] = final_world_cup['Winner'].replace(mapping)

In [ ]:
final_world_cup['Length'] = final_world_cup['Winner'].str.len()

In [ ]:
list_ = final_world_cup['Winner'].value_counts()

In [ ]:
unique_values = final_world_cup['Winner'].unique()
print(unique_values)

['New Zealand ' 'England' 'Bangladesh ' 'South Africa' 'New Zealand'
 'South Africa ' 'Australia' 'Pakistan' 'India' 'Sri Lanka' 'Bangladesh'
 'Afghanistan']


In [ ]:
final_world_cup['Winner'] = final_world_cup['Winner'].replace('Bangladesh ', 'Bangladesh')
final_world_cup['Winner'] = final_world_cup['Winner'].replace('New Zealand ', 'New Zealand')
final_world_cup['Winner'] = final_world_cup['Winner'].replace('South Africa ', 'South Africa')

In [ ]:
final_world_cup['Winner'].value_counts()

England         10
New Zealand      8
India            8
Pakistan         7
Sri Lanka        7
South Africa     6
Bangladesh       3
Australia        3
Afghanistan      3
Name: Winner, dtype: int64

In [ ]:
final_world_cup['Winner'].value_counts()

England          10
India             8
New Zealand       7
Pakistan          7
Sri Lanka         7
South Africa      5
Australia         3
Afghanistan       3
Bangladesh        2
New Zealand       1
Bangladesh        1
South Africa      1
Name: Winner, dtype: int64

In [ ]:
final_world_cup.head()

,Team_1,Team_2,Winner,Batting,Year,Team_1_NRR,Team_1_Pts,Team_2_NRR,Team_2_Pts,Ground_encoded,Team_1_encoded,Team_2_encoded,winner_encoded
0,England,New Zealand,New Zealand,1,2015,-0.753,4.0,2.564,12.0,13,3,6,New Zealand
1,England,Australia,England,1,2015,-0.753,4.0,2.564,12.0,19,3,1,3
2,Bangladesh,India,Bangladesh,1,2015,0.136,7.0,1.827,12.0,28,2,4,Bangladesh
3,South Africa,New Zealand,South Africa,1,2015,1.707,8.0,2.257,9.0,29,8,6,8
4,Pakistan,England,England,1,2015,-0.085,8.0,-0.753,4.0,5,7,3,3


In [ ]:
final_world_cup.drop(['winner_encoded'],axis=1,inplace=True)

final_world_cup['winner_encoded'] = final_world_cup['Winner'].replace(mapping)

In [ ]:
final_world_cup.head()

,Team_1,Team_2,Winner,Batting,Year,Team_1_NRR,Team_1_Pts,Team_2_NRR,Team_2_Pts,Ground_encoded,Team_1_encoded,Team_2_encoded,Length,winner_encoded
0,England,New Zealand,New Zealand,1,2015,-0.753,4.0,2.564,12.0,13,3,6,12,6
1,England,Australia,England,1,2015,-0.753,4.0,2.564,12.0,19,3,1,7,3
2,Bangladesh,India,Bangladesh,1,2015,0.136,7.0,1.827,12.0,28,2,4,11,2
3,South Africa,New Zealand,South Africa,1,2015,1.707,8.0,2.257,9.0,29,8,6,12,8
4,Pakistan,England,England,1,2015,-0.085,8.0,-0.753,4.0,5,7,3,7,3


In [ ]:
final_world_cup.drop(['Team_1','Team_2','Winner'],axis=1,inplace=True)

In [ ]:
final_world_cup.head()

,Batting,Year,Team_1_NRR,Team_1_Pts,Team_2_NRR,Team_2_Pts,Ground_encoded,Team_1_encoded,Team_2_encoded,Length,winner_encoded
0,1,2015,-0.753,4.0,2.564,12.0,13,3,6,12,6
1,1,2015,-0.753,4.0,2.564,12.0,19,3,1,7,3
2,1,2015,0.136,7.0,1.827,12.0,28,2,4,11,2
3,1,2015,1.707,8.0,2.257,9.0,29,8,6,12,8
4,1,2015,-0.085,8.0,-0.753,4.0,5,7,3,7,3


In [ ]:
final_world_cup = pd.read_csv('/content/drive/Shareddrives/Repository/data_mining_project/final_icc_worldcup_11_date.csv')

final_world_cup.columns

Index(['Team_1', 'Team_2', 'Winner', 'Batting', 'Year', 'Team_1_NRR',
       'Team_1_Pts', 'Team_2_NRR', 'Team_2_Pts', 'Ground_encoded',
       'Team_1_encoded', 'Team_2_encoded', 'Length', 'winner_encoded'],
      dtype='object')

In [ ]:
final_world_cup.loc[final_world_cup.Winner == final_world_cup.Team_1,'winning_team']=1

final_world_cup.loc[final_world_cup.Winner == final_world_cup.Team_2, 'winning_team']=2

final_world_cup['winning_team'].value_counts()

1.0    38
2.0    17
Name: winning_team, dtype: int64

In [ ]:
final_world_cup.drop(['Length','Team_1','Team_2','Winner','winner_encoded'],axis=1,inplace=True)

# **Training and testing**

In [ ]:
from sklearn.model_selection import train_test_split

df_train = final_world_cup.drop(["winning_team"],axis=1)
df_test = final_world_cup['winning_team']

X_train, X_test, y_train, y_test = train_test_split(df_train, df_test, test_size=0.2, random_state=72)

In [ ]:
!pip install xgboost

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [ ]:
print(dtrain)

In [ ]:
params = {
    'objective': 'multi:softmax',
    'num_class': 10,
    'max_depth': 12,
    'learning_rate': 0.1,
    'eval_metric': 'mlogloss'
}

num_rounds = 100
model = xgb.train(params, dtrain, num_rounds)

y_pred = model.predict(dtest)


accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print('Classification Report:\n', classification_rep)

Accuracy: 0.7272727272727273
Classification Report:
               precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
           3       1.00      0.50      0.67         2
           4       1.00      1.00      1.00         1
           6       0.50      1.00      0.67         1
           7       0.33      0.50      0.40         2
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         2

    accuracy                           0.73        11
   macro avg       0.69      0.71      0.68        11
weighted avg       0.74      0.73      0.71        11



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
params = {
    'objective': 'multi:softmax',
    'num_class': 2,
    'max_depth': 12,
    'learning_rate': 0.1,
    'eval_metric': 'mlogloss'
}

num_rounds = 100
model = xgb.train(params, dtrain, num_rounds)

y_pred = model.predict(dtest)


accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print('Classification Report:\n', classification_rep)

XGBoostError: ignored

## **CatBoostClassifier**

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.1 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


# Initialize the CatBoost classifier
catboost_classifier = CatBoostClassifier(iterations=100, depth=15, learning_rate=0.001, loss_function='MultiClass')

# Train the CatBoost model
catboost_classifier.fit(X_train, y_train, cat_features=[])

# Predict the labels on the test set
y_pred = catboost_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:\n', classification_rep)

0:	learn: 2.1967687	total: 5.95ms	remaining: 589ms
1:	learn: 2.1962985	total: 18.5ms	remaining: 909ms
2:	learn: 2.1958946	total: 24ms	remaining: 775ms
3:	learn: 2.1955433	total: 27.7ms	remaining: 665ms
4:	learn: 2.1949860	total: 28.3ms	remaining: 538ms
5:	learn: 2.1946374	total: 67.2ms	remaining: 1.05s
6:	learn: 2.1940803	total: 69.1ms	remaining: 918ms
7:	learn: 2.1936517	total: 70.8ms	remaining: 815ms
8:	learn: 2.1931743	total: 71.7ms	remaining: 725ms
9:	learn: 2.1926484	total: 81.7ms	remaining: 735ms
10:	learn: 2.1921416	total: 83.1ms	remaining: 672ms
11:	learn: 2.1917923	total: 103ms	remaining: 754ms
12:	learn: 2.1914207	total: 113ms	remaining: 754ms
13:	learn: 2.1909838	total: 116ms	remaining: 710ms
14:	learn: 2.1905499	total: 125ms	remaining: 710ms
15:	learn: 2.1901321	total: 127ms	remaining: 665ms
16:	learn: 2.1898021	total: 136ms	remaining: 665ms
17:	learn: 2.1891619	total: 137ms	remaining: 624ms
18:	learn: 2.1888178	total: 140ms	remaining: 597ms
19:	learn: 2.1884427	total: 141m

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

## **LGBMClassifier**

In [ ]:
!pip install lightgbm

In [ ]:
from lightgbm import LGBMClassifier

# Initialize the LGBMClassifier
lgb_classifier = LGBMClassifier(n_estimators=300, learning_rate=0.1, max_depth=20)

# Train the LGBM model
lgb_classifier.fit(X_train, y_train)

# Predict the labels on the test set
y_pred = lgb_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:\n', classification_rep)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000040 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 57
[LightGBM] [Info] Number of data points in the train set: 44, number of used features: 6
[LightGBM] [Info] Start training from score -2.685577
[LightGBM] [Info] Start training from score -3.091042
[LightGBM] [Info] Start training from score -2.685577
[LightGBM] [Info] Start training from score -1.704748
[LightGBM] [Info] Start training from score -1.838279
[LightGBM] [Info] Start training from score -1.838279
[LightGBM] [Info] Start training from score -2.174752
[LightGBM] [Info] Start training from score -2.397895
[LightGBM] [Info] Start t

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create an Random forest classifier with a linear kernel
rf = RandomForestClassifier(n_estimators=100, max_depth=20,
                              random_state=0)

# Fit the classifier on the training data
rf.fit(X_train, y_train)

# Make predictions on the test data
pred = rf.predict(X_test)

# Calculating training set score:
score = rf.score(X_train, y_train)

# Calculating test set score:
score2 = rf.score(X_test, y_test)

# Printing the training set accuracy with 3 decimal places
print("Training set accuracy: ", '%.3f'%(score))

# Printing the test set accuracy with 3 decimal places
print("Test set accuracy: ", '%.3f'%(score2))

Training set accuracy:  0.955
Test set accuracy:  0.727


## **Deep Neural Network**

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.callbacks import ModelCheckpoint

# Assuming you have a pandas DataFrame 'df' with features and labels
# Replace 'features_column_names' and 'label_column_name' with your actual column names

# Separate features and labels
# df_train = final_world_cup.drop(['winner_encoded'],axis=1)
X = df_train.values
y = df_test.values

# Convert labels to one-hot encoding
y_one_hot = tf.keras.utils.to_categorical(y, num_classes=10)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build the DNN model using Keras
model = tf.keras.Sequential([
        tf.keras.layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')   # Adjust the number of units for the number of classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint_callback = ModelCheckpoint(filepath='/content/drive/Shareddrives/Repository/data_mining_project/icc_best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Predict labels on the test set
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

# Evaluate the model
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
classification_rep = classification_report(np.argmax(y_test, axis=1), y_pred)

print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:\n', classification_rep)

Epoch 1/100
2/2 [==============================] - 1s 212ms/step - loss: 2.2843 - accuracy: 0.1364 - val_loss: 2.2314 - val_accuracy: 0.4545
Epoch 2/100
2/2 [==============================] - 0s 38ms/step - loss: 2.2475 - accuracy: 0.1818 - val_loss: 2.1801 - val_accuracy: 0.3636
Epoch 3/100
2/2 [==============================] - 0s 42ms/step - loss: 2.1600 - accuracy: 0.2955 - val_loss: 2.1327 - val_accuracy: 0.4545
Epoch 4/100
2/2 [==============================] - 0s 40ms/step - loss: 2.1003 - accuracy: 0.3864 - val_loss: 2.0762 - val_accuracy: 0.4545
Epoch 5/100
2/2 [==============================] - 0s 42ms/step - loss: 2.0549 - accuracy: 0.4318 - val_loss: 2.0118 - val_accuracy: 0.5455
Epoch 6/100
2/2 [==============================] - 0s 57ms/step - loss: 2.0100 - accuracy: 0.3864 - val_loss: 1.9464 - val_accuracy: 0.5455
Epoch 7/100
2/2 [==============================] - 0s 55ms/step - loss: 1.9247 - accuracy: 0.4318 - val_loss: 1.8753 - val_accuracy: 0.4545
Epoch 8/100
2/2 [==

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [ ]:
df_train.columns

Index(['Batting', 'Year', 'Team_1_NRR', 'Team_1_Pts', 'Team_2_NRR',
       'Team_2_Pts', 'Ground_encoded', 'Team_1_encoded', 'Team_2_encoded',
       'Length'],
      dtype='object')

# **Hyperparameter tuning**

## **Preparing training and testing data**

In [ ]:
icc_df = pd.read_csv('/content/drive/Shareddrives/Repository/data_mining_project/final_icc_worldcup_11_date.csv')
icc_df.columns

In [ ]:
icc_df.drop(['Team_1','Team_2','Winner'],axis =1,inplace=True)

In [ ]:
X = icc_df.drop(['winner_encoded'],axis=1)
y = icc_df['winner_encoded']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=87)

In [ ]:
y_test.value_counts()

3    2
4    2
8    2
1    1
7    1
0    1
2    1
9    1
Name: winner_encoded, dtype: int64

## **Catboost**

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 2.3 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Define the parameter grid to search
param_grid = {
    'iterations': [100, 150, 200],
    'learning_rate': [0.01, 0.001, 0.1],
    'depth': [3, 5, 7],
    'l2_leaf_reg': [1, 3, 5],
    'border_count': [32, 64, 128],
}

# Initialize the CatBoost classifier for multiclass classification
catboost_model = CatBoostClassifier()

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=catboost_model, param_grid=param_grid, scoring='accuracy', cv=3)

# Fit the model to the training data
grid_search.fit(X_train, y_train)

# Get the best parameters and best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("Best Parameters:", best_params)
print("Test Accuracy:", accuracy)
print("Classification Report:\n", classification_rep)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


Streaming output truncated to the last 5000 lines.
19:	learn: 2.1743613	total: 33.9ms	remaining: 305ms
20:	learn: 2.1732289	total: 35.4ms	remaining: 302ms
21:	learn: 2.1720819	total: 36.9ms	remaining: 299ms
22:	learn: 2.1707718	total: 37.6ms	remaining: 289ms
23:	learn: 2.1696445	total: 38.2ms	remaining: 280ms
24:	learn: 2.1680402	total: 38.9ms	remaining: 272ms
25:	learn: 2.1668261	total: 39.9ms	remaining: 267ms
26:	learn: 2.1657267	total: 41.4ms	remaining: 265ms
27:	learn: 2.1646935	total: 43ms	remaining: 264ms
28:	learn: 2.1637311	total: 44.4ms	remaining: 262ms
29:	learn: 2.1625965	total: 45.1ms	remaining: 256ms
30:	learn: 2.1619404	total: 46ms	remaining: 251ms
31:	learn: 2.1605072	total: 47.3ms	remaining: 248ms
32:	learn: 2.1597859	total: 47.9ms	remaining: 242ms
33:	learn: 2.1587663	total: 49ms	remaining: 239ms
34:	learn: 2.1578376	total: 49.5ms	remaining: 233ms
35:	learn: 2.1569802	total: 51.1ms	remaining: 233ms
36:	learn: 2.1560562	total: 52.7ms	remaining: 232ms
37:	learn: 2.154466

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

# **Training and testing**

In [ ]:
final_world_cup['winning_team'].replace({1: 0, 2: 1}, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split

df_train = final_world_cup.drop(["winning_team"],axis=1)
df_test = final_world_cup['winning_team']

X_train, X_test, y_train, y_test = train_test_split(df_train, df_test, test_size=0.2, random_state=72)

In [ ]:
!pip install xgboost

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [ ]:
print(dtrain)

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {
    'objective': 'binary:logistic',
    'max_depth':20,
    'learning_rate': 0.01,
    'eval_metric': 'logloss'
}


num_rounds = 1000
model = xgb.train(params, dtrain, num_rounds)

y_pred = model.predict(dtest)

threshold = 0.5  # adjust as needed
y_pred_binary = [1 if pred > threshold else 0 for pred in y_pred]

accuracy = accuracy_score(y_test, y_pred_binary)
print(f"Accuracy: {accuracy}")
classification_rep = classification_report(y_test, y_pred_binary)

print('Classification Report:\n', classification_rep)

Accuracy: 0.6363636363636364
Classification Report:
               precision    recall  f1-score   support

         0.0       0.75      0.75      0.75         8
         1.0       0.33      0.33      0.33         3

    accuracy                           0.64        11
   macro avg       0.54      0.54      0.54        11
weighted avg       0.64      0.64      0.64        11



## **CatBoostClassifier**

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.9 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


# Initialize the CatBoost classifier
# Initialize CatBoostClassifier
model = CatBoostClassifier(iterations=200, depth=15, learning_rate=0.01, loss_function='Logloss')

# Train the model
model.fit(X_train, y_train, cat_features=[])

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

print("Classification Report:")
print(classification_report(y_test, y_pred))

# print("Confusion Matrix:")
# print(confusion_matrix(y_test, y_pred))


0:	learn: 0.6865117	total: 13.4ms	remaining: 2.67s
1:	learn: 0.6798258	total: 21ms	remaining: 2.08s
2:	learn: 0.6745807	total: 26.5ms	remaining: 1.74s
3:	learn: 0.6681218	total: 28.2ms	remaining: 1.38s
4:	learn: 0.6623073	total: 30.6ms	remaining: 1.19s
5:	learn: 0.6590304	total: 32.9ms	remaining: 1.06s
6:	learn: 0.6542448	total: 36.8ms	remaining: 1.01s
7:	learn: 0.6497554	total: 41.7ms	remaining: 1s
8:	learn: 0.6449972	total: 44.1ms	remaining: 935ms
9:	learn: 0.6392871	total: 49.2ms	remaining: 936ms
10:	learn: 0.6348165	total: 51.6ms	remaining: 886ms
11:	learn: 0.6294871	total: 53.3ms	remaining: 834ms
12:	learn: 0.6268106	total: 54.5ms	remaining: 784ms
13:	learn: 0.6227372	total: 57.8ms	remaining: 768ms
14:	learn: 0.6184344	total: 62ms	remaining: 764ms
15:	learn: 0.6140620	total: 64.4ms	remaining: 740ms
16:	learn: 0.6096731	total: 66.3ms	remaining: 714ms
17:	learn: 0.6048320	total: 68.7ms	remaining: 695ms
18:	learn: 0.5999722	total: 71.6ms	remaining: 682ms
19:	learn: 0.5957866	total: 7

## **LGBMClassifier**

In [ ]:
!pip install lightgbm

In [ ]:
from lightgbm import LGBMClassifier

# Initialize the LGBMClassifier
lgb_classifier = LGBMClassifier(n_estimators=300, learning_rate=0.1, max_depth=20)

# Train the LGBM model
lgb_classifier.fit(X_train, y_train)

# Predict the labels on the test set
y_pred = lgb_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:\n', classification_rep)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 14, number of negative: 30
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000277 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 57
[LightGBM] [Info] Number of data points in the train set: 44, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.318182 -> initscore=-0.762140
[LightGBM] [Info] Start training from score -0.762140
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with 

## **Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create an Random forest classifier with a linear kernel
rf = RandomForestClassifier(n_estimators=100, max_depth=20,
                              random_state=0)

# Fit the classifier on the training data
rf.fit(X_train, y_train)

# Make predictions on the test data
pred = rf.predict(X_test)

# Calculating training set score:
score = rf.score(X_train, y_train)

# Calculating test set score:
score2 = rf.score(X_test, y_test)

# Printing the training set accuracy with 3 decimal places
print("Training set accuracy: ", '%.3f'%(score))

# Printing the test set accuracy with 3 decimal places
print("Test set accuracy: ", '%.3f'%(score2))

Training set accuracy:  0.955
Test set accuracy:  0.636


## **Deep Neural Network**

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.callbacks import ModelCheckpoint

# Assuming you have a pandas DataFrame 'df' with features and labels
# Replace 'features_column_names' and 'label_column_name' with your actual column names

# Separate features and labels
# df_train = final_world_cup.drop(['winner_encoded'],axis=1)
X = df_train.values
y = df_test.values

# Convert labels to one-hot encoding
# y_one_hot = tf.keras.utils.to_categorical(y, num_classes=10)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build the DNN model using Keras
model = tf.keras.Sequential([
        tf.keras.layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')   # Adjust the number of units for the number of classes
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


checkpoint_callback = ModelCheckpoint(filepath='/content/drive/Shareddrives/Repository/data_mining_project/icc_best_model_binary.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Predict labels on the test set
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)


# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:\n', classification_rep)

Epoch 1/100
2/2 [==============================] - 4s 280ms/step - loss: 0.6723 - accuracy: 0.6136 - val_loss: 0.6073 - val_accuracy: 0.9091
Epoch 2/100
2/2 [==============================] - 0s 37ms/step - loss: 0.6526 - accuracy: 0.6591 - val_loss: 0.5343 - val_accuracy: 0.9091
Epoch 3/100
2/2 [==============================] - 0s 70ms/step - loss: 0.6357 - accuracy: 0.6364 - val_loss: 0.4760 - val_accuracy: 0.9091
Epoch 4/100
2/2 [==============================] - 0s 104ms/step - loss: 0.5913 - accuracy: 0.6591 - val_loss: 0.4255 - val_accuracy: 0.9091
Epoch 5/100
2/2 [==============================] - 0s 181ms/step - loss: 0.6031 - accuracy: 0.6818 - val_loss: 0.3799 - val_accuracy: 0.9091
Epoch 6/100
2/2 [==============================] - 0s 165ms/step - loss: 0.5780 - accuracy: 0.6364 - val_loss: 0.3374 - val_accuracy: 0.9091
Epoch 7/100
2/2 [==============================] - 0s 129ms/step - loss: 0.5770 - accuracy: 0.6818 - val_loss: 0.3037 - val_accuracy: 0.9091
Epoch 8/100
2/2